In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader, random_split, ConcatDataset
import numpy as np
from datasets import *
from rnn import *
from vae import *
from losses import *
import os
from ranger21 import Ranger21
from torch.utils.tensorboard import SummaryWriter
import imageio
set_random_seed(7)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2023-06-06 15:46:41.436741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 15:46:42.168923: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
oids, labels = get_only_r_oids('akb.ztf.snad.space.json')
data = EmbsSequenceData(oids, labels, return_oid=True)
bucket_boundaries = [200, 400, 600, 800]

sampler = BySequenceLengthSampler(data, bucket_boundaries, 32, drop_last=False, shuffle=False, return_oid=True)
loader = DataLoader(data, batch_size=1, 
                        batch_sampler=sampler, 
                        num_workers=8,
                        collate_fn=collate_with_oid,
                        drop_last=False, pin_memory=False)


In [3]:
clf = RBclassifier(hidden_size=128, latent_dim=36, rnn_type='GRU', out_size=2)
clf.load_state_dict(torch.load(f'trained_models/rnn/baseline/model.zip'))
clf.eval()


enc = VAEEncoder(latent_dim=36 * 2)
enc.load_state_dict(torch.load('trained_models/vae/encoder_100ep.zip'))
enc.eval()
for seq in enc.encoder.children():
    for child in seq.children():
        if isinstance(child, nn.BatchNorm2d):
            child.track_running_stats = False

In [4]:
fp_oids, fn_oids = [], []
fp_prob, fn_prob = [], []
for embs, labs, oids in loader:
    with torch.no_grad():
        output = clf(embs)
        prob = nn.Softmax(dim=1)(output)[:, 1]
        fp_mask = (labs == 0) & (prob > 0.5)
        fn_mask = (labs == 1) & (prob < 0.5)
    fp_oids.append(np.array(oids)[fp_mask])
    fn_oids.append(np.array(oids)[fn_mask])
    fp_prob.append(prob[fp_mask])
    fn_prob.append(prob[fn_mask])

fp_prob, fn_prob = torch.hstack(fp_prob), torch.hstack(fn_prob)
fp_oids, fn_oids = np.hstack(fp_oids), np.hstack(fn_oids)

In [5]:
fp_prob_sort, indx_fp = torch.sort(fp_prob)

In [6]:
fn_prob_sort, indx_fn = torch.sort(fn_prob)

In [14]:
oid = fn_oids[indx_fn[1]]

In [15]:
fr = get_frames_seq(f'data/{oid}').view(-1, 28, 28, 1)
imageio.mimsave(f'gifs/{oid}.gif', # output gif
                fr,          # array of input frames
                fps = 10)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

In [16]:
from IPython.display import display, Image
Image(f'gifs/{oid}.gif', width=150, height=150)

In [17]:
oid

824214200002554